## 6. Calculations for comparison paper

## Setting things up
First we need to import a few Python packages (including VolFe) and set the precision of the calculations. You need to install VolFe once on your machine, if you haven't yet, uncomment the line below (remove the #).

In [2]:
# Install VolFe on your machine. Don't remove the # from this line!
# pip install VolFe # Remove the first # in this line if you have not installed VolFe on your machine before.

# import python packages
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import VolFe as vf

In [3]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Kīlauea_Dryend',
           'T_C': 1220., # Temperature in 'C
           'SiO2': 50.142, # wt%
           'TiO2': 2.257, # wt%
           'Al2O3': 12.224, # wt%
           'FeOT': 11.33, # wt%
           'MnO': 0.323, # wt%
           'MgO': 10.215, # wt%
           'CaO': 10.476, # wt%
           'Na2O': 1.754, # wt%
           'K2O': 0.398, # wt%
           'P2O5': 0.233, # wt%
           'H2O': 0.3, # wt%
           'CO2ppm': 800., # ppm
           'STppm': 1500., # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.18}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# Choose the options I want to change for the calculation - everything else will use the default options
my_models = [['solve_species','OHS'],['print status','True']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

In [5]:
KilaueaDry = vf.calc_gassing(my_analysis, dp_step=1., models=my_models)

Kīlauea_Dryend
1220.0 1711.9992252609893 2024-05-28 20:05:11.428790
1220.0 1612.0 0.5750623511248882  2024-05-28 20:05:25.831435
1220.0 1512.0 0.5769012790524597  2024-05-28 20:05:40.314861
1220.0 1412.0 0.5785713911160881  2024-05-28 20:05:55.325290
1220.0 1312.0 0.5800455341060422  2024-05-28 20:06:09.744979
1220.0 1212.0 0.5812878548274121  2024-05-28 20:06:25.753277
1114.0 : Switching solve species from OHS to OCS
1114.0 : Switching solve species from OCS to OCH
solver failed, calculation aborted at P =  1114.0 2024-05-28 20:06:47.609697


In [6]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Kīlauea_Wetend',
           'T_C': 1220., # Temperature in 'C
           'SiO2': 50.142, # wt%
           'TiO2': 2.257, # wt%
           'Al2O3': 12.224, # wt%
           'FeOT': 11.33, # wt%
           'MnO': 0.323, # wt%
           'MgO': 10.215, # wt%
           'CaO': 10.476, # wt%
           'Na2O': 1.754, # wt%
           'K2O': 0.398, # wt%
           'P2O5': 0.233, # wt%
           'H2O': 0.6, # wt%
           'CO2ppm': 800., # ppm
           'STppm': 1500., # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.18}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

In [7]:
KilaueaWet = vf.calc_gassing(my_analysis)

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/equilibrium_equations.py:2513: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:399: RuntimeWarning:

invalid value encountered in scalar divide

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:404: RuntimeWarning:

invalid value encountered in scalar divide

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:409: RuntimeWarning:

invalid value encountered in scalar divide

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:414: RuntimeWarning:

invalid value encountered in scalar divide

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:429: RuntimeWarning:

invalid value encountered in scalar divide



nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan enco

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:340: RuntimeWarning:

invalid value encountered in scalar divide

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:345: RuntimeWarning:

invalid value encountered in scalar divide

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:350: RuntimeWarning:

invalid value encountered in scalar divide

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:355: RuntimeWarning:

invalid value encountered in scalar divide

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:360: RuntimeWarning:

invalid value encountered in scalar divide

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:365: RuntimeWarning:

invalid value encountered in scalar divide

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/differential_equations.py:370: RuntimeWarning:

invalid value encountered in scalar divide

/Users/ehughes/Docum

nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan enco

In [8]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'MORB',
           'T_C': 1100., # Temperature in 'C
           'SiO2': 47.4, # wt%
           'TiO2': 1.01, # wt%
           'Al2O3': 17.64, # wt%
           'FeOT': 7.98, # wt%
           'MnO': 0., # wt%
           'MgO': 7.63, # wt%
           'CaO': 12.44, # wt%
           'Na2O': 2.65, # wt%
           'K2O': 0.03, # wt%
           'P2O5':0.08, # wt%
           'H2O': 0.2, # wt%
           'CO2ppm': 1100., # ppm
           'STppm': 1400., # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.1}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# Choose the options I want to change for the calculation - everything else will use the default options
my_models = [['solve_species','OHS'],['print status','True']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

In [9]:
MORB = vf.calc_gassing(my_analysis, dp_step=1., models=my_models)

MORB
1100.0 2193.9742708547615 2024-05-28 20:07:36.728932
1100.0 2094.0 -1.005932901689004 WARNING: sulfide-saturated 2024-05-28 20:07:52.152287
1100.0 1994.0 -1.0052894949696043 WARNING: sulfide-saturated 2024-05-28 20:08:06.239806
1100.0 1894.0 -1.004650307725333 WARNING: sulfide-saturated 2024-05-28 20:08:21.264063
1100.0 1794.0 -1.004014140516933 WARNING: sulfide-saturated 2024-05-28 20:08:36.364126
1100.0 1694.0 -1.003379623110856 WARNING: sulfide-saturated 2024-05-28 20:08:51.520272
1100.0 1594.0 -1.0027452129358956 WARNING: sulfide-saturated 2024-05-28 20:09:06.300960
1100.0 1494.0 -1.0021091466041643 WARNING: sulfide-saturated 2024-05-28 20:09:20.704235
1100.0 1394.0 -1.0014693739762652 WARNING: sulfide-saturated 2024-05-28 20:09:34.898229
1100.0 1294.0 -1.000823466525647 WARNING: sulfide-saturated 2024-05-28 20:09:49.052501
1100.0 1194.0 -1.0001684868124094 WARNING: sulfide-saturated 2024-05-28 20:10:03.349893
1100.0 1094.0 -0.9995007972358518 WARNING: sulfide-saturated 2024-0

In [12]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'OIB',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 51.46, # wt%
           'TiO2': 1.06, # wt%
           'Al2O3': 17.43, # wt%
           'FeOT': 9.42, # wt%
           'MnO': 0.19, # wt%
           'MgO': 3.78, # wt%
           'CaO': 7.99, # wt%
           'Na2O': 3.47, # wt%
           'K2O': 0.78, # wt%
           'P2O5': 0.24, # wt%
           'H2O': 2., # wt%
           'CO2ppm': 4000., # ppm
           'STppm': 4000., # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.3}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

In [14]:
Fuego = vf.calc_gassing(my_analysis)

Fuego
1200.0 1542.2367004237485 2024-05-29 03:59:04.628262
1200.0 2530.0 -0.45410508332089705 WARNING: sulfide-saturated 2024-05-29 03:59:31.250926
1200.0 3530.0 -0.5106372164745618 WARNING: sulfide-saturated 2024-05-29 03:59:57.509983
done 2024-05-29 04:00:10.055688


In [26]:
fig = make_subplots(rows=2, cols=3, shared_yaxes = True, shared_xaxes = False,vertical_spacing=0.08, horizontal_spacing=0.05)

lw = 3
syms = 10
lc1 = "steelblue"
lc2 = "firebrick"

data1 = KilaueaWet
data2 = KilaueaDry
data3 = Fuego_regas

r = 1
# P vs. CO2
c = 1
fig.add_trace(go.Scatter(mode = "lines", x=data1['CO2T-eq_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['CO2T-eq_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data3['CO2T-eq_ppmw'], y=data3['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "CO<sub>2</sub>-eq (ppmw)", row = r, col = c)
fig.update_yaxes(title = "<i>P</i> (bars)", autorange = 'reversed', row = r, col = c)

# P vs. H2O
c = 2
fig.add_trace(go.Scatter(mode = "lines", x=data1['H2OT-eq_wtpc'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['H2OT-eq_wtpc'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "H<sub>2</sub>O-eq (wt%)", row = r, col = c)

# P vs. S
c = 3
fig.add_trace(go.Scatter(mode = "lines", x=data1['ST_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['ST_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "S<sub>T</sub> (ppmw)", row = r, col = c)

r = 2
# P vs. S6+/ST
c = 1
fig.add_trace(go.Scatter(mode = "lines", x=data1['S6+/ST'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['S6+/ST'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data3['S6+/ST'], y=data3['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "CO<sub>2</sub>-eq (ppmw)", row = r, col = c)
fig.update_yaxes(title = "<i>P</i> (bars)", autorange = 'reversed', row = r, col = c)

# P vs. Fe3+/ST
c = 2
fig.add_trace(go.Scatter(mode = "lines", x=data1['Fe3+/FeT'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['Fe3+/FeT'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data3['Fe3+/FeT'], y=data3['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "CO<sub>2</sub>-eq (ppmw)", row = r, col = c)
fig.update_yaxes(title = "<i>P</i> (bars)", autorange = 'reversed', row = r, col = c)




fig.update_layout(height=400, width=1000, plot_bgcolor='rgb(255,255,255)' , showlegend = False)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))

fig.update_layout(font_family="Helvetica",font_color="black")

fig.show()

In [106]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Fuego',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 51.46, # wt%
           'TiO2': 1.06, # wt%
           'Al2O3': 17.43, # wt%
           'FeOT': 9.42, # wt%
           'MnO': 0.19, # wt%
           'MgO': 3.78, # wt%
           'CaO': 7.99, # wt%
           'Na2O': 3.47, # wt%
           'K2O': 0.78, # wt%
           'P2O5': 0.24, # wt%
           'H2O': 2., # wt%
           'CO2ppm': 4000., # ppm
           'STppm': 4000., # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.3,
           'final_P': 20000.,
           'initial_CO2wtpc': 2.,} # bar

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# Choose the options I want to change for the calculation - everything else will use the default options
my_models = [['gassing_direction','regas'],['bulk_composition','melt+vapor_initialCO2'],['print status','True']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

In [107]:
OIB_regas = vf.calc_gassing(my_analysis, models=my_models)

Fuego
1200.0 6417.020955417139 2024-05-29 08:45:25.349271
1200.0 7410.0 2.099717446281236  2024-05-29 08:45:56.427309
1200.0 8410.0 2.094093808842721  2024-05-29 08:46:27.441225
1200.0 9410.0 2.0953644430930556  2024-05-29 08:46:59.900124
1200.0 10410.0 2.100597881760234  2024-05-29 08:47:32.426001
1200.0 11410.0 2.107898248864373  2024-05-29 08:48:04.692548
1200.0 12410.0 2.116066630406987  2024-05-29 08:48:37.137666
1200.0 13410.0 2.1243549740897922  2024-05-29 08:49:07.943018
1200.0 14410.0 2.1322993694051258  2024-05-29 08:49:36.718926
1200.0 15410.0 2.1396122895602803  2024-05-29 08:50:05.554098
1200.0 16410.0 2.146114903131167  2024-05-29 08:50:37.160863
1200.0 17410.0 2.1516952199349095  2024-05-29 08:51:09.121246
1200.0 18410.0 2.156282404029164  2024-05-29 08:51:40.491004
1200.0 19410.0 2.1598310466036583  2024-05-29 08:52:09.747881
done 2024-05-29 08:52:27.561031


In [96]:
fig = make_subplots(rows=2, cols=3, shared_yaxes = True, shared_xaxes = False,vertical_spacing=0.08, horizontal_spacing=0.05)

lw = 3
syms = 10
lc1 = "steelblue"
lc2 = "firebrick"
lc3 = "grey"
lc4 = 'black'

data1 = Fuego_regas # 2wt% CO2 initial
data2 = Fuego_regas1 # 1 wt% CO2 initial
data3 = Fuego_regas2 # 0.125 Fe3+/FeT
data4 = Fuego_degas  

r = 1
# P vs. CO2
c = 1
fig.add_trace(go.Scatter(mode = "lines", x=data1['CO2T-eq_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['CO2T-eq_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data3['CO2T-eq_ppmw'], y=data3['P_bar'], line_color = lc3, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data4['CO2T-eq_ppmw'], y=data4['P_bar'], line_color = lc4, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "CO<sub>2</sub>-eq (ppmw)", row = r, col = c)
fig.update_yaxes(title = "<i>P</i> (bars)", autorange = 'reversed', row = r, col = c)

# P vs. H2O
c = 2
fig.add_trace(go.Scatter(mode = "lines", x=data1['H2OT-eq_wtpc'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['H2OT-eq_wtpc'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data3['H2OT-eq_wtpc'], y=data3['P_bar'], line_color = lc3, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data4['H2OT-eq_wtpc'], y=data4['P_bar'], line_color = lc4, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "H<sub>2</sub>O-eq (wt%)", row = r, col = c)

# P vs. S
c = 3
fig.add_trace(go.Scatter(mode = "lines", x=data1['ST_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['ST_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data3['ST_ppmw'], y=data3['P_bar'], line_color = lc3, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data4['ST_ppmw'], y=data4['P_bar'], line_color = lc4, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "S<sub>T</sub> (ppmw)", row = r, col = c)

r = 2
# P vs. S6+/ST
c = 1
fig.add_trace(go.Scatter(mode = "lines", x=data1['S6+/ST'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['S6+/ST'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data3['S6+/ST'], y=data3['P_bar'], line_color = lc3, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data4['S6+/ST'], y=data4['P_bar'], line_color = lc4, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "S6+/ST", row = r, col = c)
fig.update_yaxes(title = "<i>P</i> (bars)", autorange = 'reversed', row = r, col = c)

# P vs. Fe3+/ST
c = 2
fig.add_trace(go.Scatter(mode = "lines", x=data1['Fe3+/FeT'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['Fe3+/FeT'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data3['Fe3+/FeT'], y=data3['P_bar'], line_color = lc3, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data4['Fe3+/FeT'], y=data4['P_bar'], line_color = lc4, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "Fe3+/FeT", row = r, col = c)

# P vs. DFMQ
c = 3
fig.add_trace(go.Scatter(mode = "lines", x=data1['fO2_DFMQ'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['fO2_DFMQ'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data3['fO2_DFMQ'], y=data3['P_bar'], line_color = lc3, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data4['fO2_DFMQ'], y=data4['P_bar'], line_color = lc4, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "DFMQ", row = r, col = c)

fig.update_layout(height=700, width=1000, plot_bgcolor='rgb(255,255,255)' , showlegend = False)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))

fig.update_layout(font_family="Helvetica",font_color="black")

fig.show()

In [108]:
fig = make_subplots(rows=2, cols=3, shared_yaxes = True, shared_xaxes = False,vertical_spacing=0.08, horizontal_spacing=0.05)

lw = 3
syms = 10
lc1 = "steelblue"
lc2 = "firebrick"
lc3 = "grey"
lc4 = 'black'

data1 = OIB_regas # 2wt% CO2 initial
data2 = OIB_regas # 1 wt% CO2 initial

r = 1
# P vs. CO2
c = 1
fig.add_trace(go.Scatter(mode = "lines", x=data1['CO2T-eq_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['CO2T-eq_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "CO<sub>2</sub>-eq (ppmw)", row = r, col = c)
fig.update_yaxes(title = "<i>P</i> (bars)", autorange = 'reversed', row = r, col = c)

# P vs. H2O
c = 2
fig.add_trace(go.Scatter(mode = "lines", x=data1['H2OT-eq_wtpc'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['H2OT-eq_wtpc'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "H<sub>2</sub>O-eq (wt%)", row = r, col = c)

# P vs. S
c = 3
fig.add_trace(go.Scatter(mode = "lines", x=data1['ST_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['ST_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "S<sub>T</sub> (ppmw)", row = r, col = c)

r = 2
# P vs. S6+/ST
c = 1
fig.add_trace(go.Scatter(mode = "lines", x=data1['S6+/ST'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['S6+/ST'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "S6+/ST", row = r, col = c)
fig.update_yaxes(title = "<i>P</i> (bars)", autorange = 'reversed', row = r, col = c)

# P vs. Fe3+/ST
c = 2
fig.add_trace(go.Scatter(mode = "lines", x=data1['Fe3+/FeT'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['Fe3+/FeT'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "Fe3+/FeT", row = r, col = c)

# P vs. DFMQ
c = 3
fig.add_trace(go.Scatter(mode = "lines", x=data1['fO2_DFMQ'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "lines", x=data2['fO2_DFMQ'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "DFMQ", row = r, col = c)

fig.update_layout(height=700, width=1000, plot_bgcolor='rgb(255,255,255)' , showlegend = False)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))

fig.update_layout(font_family="Helvetica",font_color="black")

fig.show()